In [4]:
""" A simple filter for the workshop.
Returns 13 transients from the night of July 13,
using the selected group in ztf_public_20180713_small
"""

import glob
import fastavro

def my_filter(current_observation):
    """ A basic filter to find transients """

    # Initialize the filtering criteria
    passes_filter = False
    annotations={}
    nopointunderneath = True
    mover = True
    real = False
    positivesubtraction = False
    brightstar = False
    short = False

    # Store the information we will use in the filtering criteria
    jdstarthist = current_observation['candidate']['jdstarthist']
    jdendhist = current_observation['candidate']['jdendhist']
    isdiffpos = current_observation['candidate']['isdiffpos']
    rbscore = current_observation['candidate']['rb']
    fwhm = current_observation['candidate']['fwhm']
    nbad = current_observation['candidate']['nbad']
    m_now = current_observation['candidate']['magpsf']
    m_app = current_observation['candidate']['magap']
    psfminap = m_now - m_app
    sgscore = current_observation['candidate']['sgscore1']
    sgscore2 = current_observation['candidate']['sgscore2']
    sgscore3 = current_observation['candidate']['sgscore3']
    distpsnr1 = current_observation['candidate']['distpsnr1']
    srmag = current_observation['candidate']['srmag1']
    distpsnr2 = current_observation['candidate']['distpsnr2']
    distpsnr3 = current_observation['candidate']['distpsnr3']
    srmag2 = current_observation['candidate']['srmag2']
    srmag3 = current_observation['candidate']['srmag3']
    prevcandidates = current_observation['prv_candidates']
    t_now = current_observation['candidate']['jd']

    # Total duration of the transient < 20 days
    if (jdstarthist and jdendhist):
        deltajd = jdendhist - jdstarthist
        if (deltajd < 20):
            short = True

    # Make sure it is a positive subtraction
    if (isdiffpos and (isdiffpos == 't' or isdiffpos == '1')):
        positivesubtraction = True

    # Make sure it is real, and not bogus
    if (rbscore and rbscore > 0.3 and fwhm > 0.5 and nbad < 5 and (psfminap < 0.75 or psfminap > (-0.75))):
        real = True

    # Make sure it is not within 2 arcseconds of a star
    if (sgscore and distpsnr1 and sgscore > 0.76 and distpsnr1 < 2):
        nopointunderneath = False

    # Make sure it is not an artifact from a bright star
    if ((distpsnr1 and srmag and distpsnr1 < 20 and srmag < 15.0 and srmag > 0 and sgscore > 0.49) or (distpsnr2 and srmag2 and distpsnr2 < 20 and srmag2 < 15.0 and srmag2 > 0 and sgscore2 > 0.49) or (distpsnr3 and srmag3 and distpsnr3 < 20 and srmag3 < 15.0 and srmag3 > 0 and sgscore3 > 0.49)):
        brightstar = True

    if (prevcandidates):
        # Loop through all previous candidates at this location
        for candidate in prevcandidates:
            if (
                candidate['jd'] and candidate['magpsf'] and candidate['isdiffpos'] 
                and (candidate['isdiffpos'] == '1' or candidate['isdiffpos'] == 't')):
                # calculate the change in time between this previous candidate
                # and the most recent one
                dt = t_now - candidate['jd']
                # to filter out asteroids, require at least two detections
                # spaced by 0.02 days (half an hour)
                if (dt > 0.02 and candidate['magpsf'] < 99):
                    mover = False

    # return some annotations 
    annotations['jd'] = t_now
    annotations['jdstarthist'] = jdstarthist
    annotations['rb score'] = rbscore
    annotations['sgscore1'] = sgscore
    annotations['distpsnr1'] = distpsnr1
    passes_filter = nopointunderneath and ((not mover)) and real and positivesubtraction and ((not brightstar)) and short

    # return the results of the filtering process
    return passes_filter, annotations 

In [5]:
files = glob.glob("ztf_public_20180713_small/*.avro")
for f in files:
    with open(f, 'rb') as fo:
        reader = fastavro.reader(fo)
        candidate = next(reader, None)
        result, annotations = my_filter(candidate)
        if result:
             print(candidate['objectId'])
